
![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/public/SENTIMENT_UR.ipynb)

# **Analyze sentiment in Urdu movie reviews and tweets**

# **Colab Setup**

In [ ]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.1.2 spark-nlp==3.4.4

# Install Spark NLP Display lib
! pip install --upgrade -q spark-nlp-display

# **Start the Spark session**

In [2]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *

import pandas as pd

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

import string
import numpy as np


spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

Spark NLP version 3.4.4
Apache Spark version: 3.1.2


# **🔎 For about model**


📌 **"sentimentdl_urduvec_imdb"**--> *Analyse sentiment in reviews by classifying them as positive or negative. This model is trained using urduvec_140M_300d word embeddings. The word embeddings are then converted to sentence embeddings before feeding to the sentiment classifier which uses a DL architecture to classify sentences.*

*  positive , negative





# **🔎Define Spark NLP pipeline**

In [19]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence_detector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")

word_embeddings = WordEmbeddingsModel()\
    .pretrained('urduvec_140M_300d', 'ur')\
    .setInputCols(["sentence",'token'])\
    .setOutputCol("word_embeddings")

sentence_embeddings = SentenceEmbeddings() \
    .setInputCols(["sentence", "word_embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

classifier = SentimentDLModel.pretrained('sentimentdl_urduvec_imdb', 'ur' )\
    .setInputCols(['sentence_embeddings'])\
    .setOutputCol('sentiment')

nlpPipeline = Pipeline(stages=[ document_assembler,
                                sentence_detector, 
                                tokenizer,
                                word_embeddings,
                                sentence_embeddings,
                                classifier
                                 ])

empty_df = spark.createDataFrame([['']]).toDF("text")
pipelineModel = nlpPipeline.fit(empty_df)
light_model = LightPipeline(pipelineModel)

urduvec_140M_300d download started this may take some time.
Approximate size to download 110.2 MB
[OK!]
sentimentdl_urduvec_imdb download started this may take some time.
Approximate size to download 9 MB
[OK!]


# **🔎Sample text**

In [4]:
text = """
    یہ کردار زندہ دل اور دلچسپ تھے ، پلاٹ بہترین طور پر چل رہا تھا ، پائرو اثرات کو مہارت کے ساتھ انجام دیا گیا تھا ، اور اس میں ایک بنیادی محبت کی مثلث کی کہانی لی گئی ہے اور اس میں سائنس فکشن عنصر کو پھینک دیا گیا ہے۔ میں بہت سارے کرداروں کے ساتھ ان کی نشاندہی کرسکتا تھا اور ان کے محرکات نے کہانی کے فریم ورک میں منطقی عقلی احساس پیدا کیا تھا۔ کیمرا کام بہت اچھا تھا ، آڈیو واضح اور درست تھا ، بیک گراؤنڈ میوزک کو اثر انداز کے لئے منتخب کیا گیا تھا ، گانے والے فائر مین ایک اچھے باصلاحیت یادگار تھے۔ عجیب کیفیت ، سیٹوں نے بہت عمدہ تیار کیا ، اور ہنر مند ہنر کے ساتھ انجام دیئے گئے خاص اثرات۔ میں حیرت زدہ ہوں کہ کس طرح چین اسٹور کی پارکنگ میں پوری منی کارنیوال کو ایک ہی لیمپپوسٹ آؤٹ لیٹ کے ذریعہ تقویت مل سکتی ہے۔ کم سے کم کہنا ناممکن لگتا ہے۔ فلم کے اختتام کے قریب بھائیوں کے مابین لڑائی شاندار تھی۔ جم ورنی کو غیر جوکر والے کردار میں رکھنا بھی ایک حیرت انگیز ٹچ تھا کیونکہ اس نے ایک کارنی کا نیم سنجیدہ کردار بہت عمدہ ادا کیا تھا۔"""

In [5]:
df = spark.createDataFrame([[text]]).toDF('text')
df.show()

+--------------------+
|                text|
+--------------------+
|
    یہ کردار زند...|
+--------------------+



# **🔎Run the pipeline**

In [6]:
result = pipelineModel.transform(df)

In [9]:
result.select('sentiment.result', 'sentence.result').show()

+----------+--------------------+
|    result|              result|
+----------+--------------------+
|[positive]|[یہ کردار زندہ دل...|
+----------+--------------------+

